<a href="https://colab.research.google.com/github/KalikaKay/Author-Classification-Project/blob/master/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning



In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.summarization import keywords
import nltk
import re
import requests
import os

!pip install rake_nltk --quiet
from rake_nltk import Rake
nltk.download('punkt')
nltk.download('wordnet')

import pandas as pd
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import spacy


# file location
PATH = '/content/drive/MyDrive/Author Classification/AuthorTexts'
DOC_PATTERN = r'.*\.txt'

corpus = PlaintextCorpusReader(PATH, DOC_PATTERN)

def corpus_stats(corpus):
  print("corpus statistics:")
  print(f'Number of Documents: {str(len(corpus.fileids()))}')
  print(f'Number of paragraphs: {str(len(corpus.paras()))}')
  print(f'Number of sentences: {str(len(corpus.sents()))}')
  print(f'Number of words: {str(len(corpus.words()))}')
  print(f'Vocabulary: {str(len(set(w.lower() for w in corpus.words())))}')
  print(f'avg chars per word: {str(round(len(corpus.raw())/len(corpus.words()), 1))}')
  print(f'avg words per sentence: {str(round(len(corpus.words())/len(corpus.sents()), 1))}')

corpus_stats(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
corpus statistics:
Number of Documents: 9
Number of paragraphs: 295
Number of sentences: 295
Number of words: 854345
Vocabulary: 23747
avg chars per word: 5.9
avg words per sentence: 2896.1


In [ ]:
#Dataframe of sentences and authors.
for fileid in corpus.fileids():
  book = pd.DataFrame(corpus.raw(fileids=fileid).split('\n'), columns=['sentence'])
  book['author'] = fileid.split('/')[0]
  try:
    books = pd.concat([books, book])
  except NameError: 
    books = book

In [ ]:
#Remove empty strings.
books = books.replace(r'^\s*$', np.NaN, regex=True)
books.dropna(inplace=True)
books.reindex()

#Remove contents and Chapter Titles.
i = 0
for sent in books.sentence:
  if any(c.islower() for c in sent) == False:
    books.iloc[i] = np.NaN
  if 'project gutenberg' in sent.lower():
    books.iloc[i] = np.NaN    
  if 'contents' == sent.lower().strip():
    books.iloc[i] == np.NaN
  i += 1

books.dropna(inplace=True)
books.reindex()

,sentence,author
0,Produced David Price,Anne Bronte
1,Agnes Grey NOVEL,Anne Bronte
5,Illustration Birthplace Charlotte Emily and An...,Anne Bronte
6,Contents,Anne Bronte
9,All true histories contain instruction though ...,Anne Bronte
...,...,...
2452,Dear Editor,Various
2453,have read the first copy Astounding Stories an...,Various
2454,Congratulations,Various
2455,Dear Editor,Various


In [ ]:
content_index = books[books.sentence == 'Contents '].index.values
books.drop(content_index, inplace=True)

In [ ]:
#Number of characters in each sentence, per author
books['length'] = [len(sen) for sen in books.sentence]
books.groupby(by='author').describe()

length              ...                 
                              count        mean  ...     75%      max
author                                           ...                 
Anne Bronte                   934.0  332.932548  ...  498.75   3299.0
Bell AKA Bronte Sisters       884.0  167.628959  ...  198.00   2741.0
Charlotte Bronte              110.0  391.136364  ...  276.50   4965.0
Edith Rickert & Gleb Botkin   191.0   60.717277  ...   79.00    381.0
Emily Bronte                 1922.0  278.165973  ...  369.00   3301.0
Ethel M. Dell                8629.0  145.794994  ...  181.00   2160.0
Fyodor Dostoevsky            5574.0  266.101722  ...  274.75  13267.0
Jane Austen                  2675.0  333.707290  ...  410.50  12284.0
Various                      2204.0  157.271325  ...  221.00   1119.0

[9 rows x 8 columns]

# Feature Engineering

These features will be fed into a pipeline for our models. 

Due to memory constraints, I've opted to visualize the vectors in separate notebooks. 

In [ ]:
#Tokenize the data
tokenizer = RegexpTokenizer(r'\w+')
books['tokenized'] = [tokenizer.tokenize(sent.lower()) for sent in books.sentence 
                   if tokenizer.tokenize(sent.lower()) not in stopwords.words('english')] 

In [ ]:
# Lemmatize the tokens. 
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
stemmed = []
lemmatized = []
for token in books.tokenized:
  stemmed.append([stemmer.stem(word) for word in token])
  lemmatized.append([lemmatizer.lemmatize(word) for word in token])
books['stemmed'] = stemmed
books['lemmatized'] = lemmatized

## Visual Representations (tables) of the vectors. 

In [ ]:
#Bag of Words
s = books['lemmatized'] #make it shorter
vectorizer = CountVectorizer(binary=True, strip_accents="ascii", stop_words="english")
vectors = vectorizer.fit_transform(s.astype(str))

#quick visualization of a vector
one_hot = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
one_hot.head()

,000,100,108,10th,113,11th,1200,12th,1399,13th,1431,14th,1500,1509,1586,15th,16th,1700,1726,1778,1790,1791,1792,1801,1802,1829,1849,1859,1861,1864,1868,1879,1880,18th,1900,1910,1930,1935,19th,1st,...,youth,youthful,youthfulness,yr,yulian,yushin,yust,yusupov,zada,zaharovitch,zametov,zamm,zapiski,zara,zarnitsyn,zeal,zealand,zealous,zealously,zebra,zed,zeitschrift,zenith,zephyr,zero,zest,zeus,zhukovsky,zigzag,zigzagged,zillah,zimmerman,zimoveykin,zinc,zinovy,zone,zoomed,zorn,zossimov,zverkov
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#TF-IDF
l = books.lemmatized.astype(str)
vectorizer = TfidfVectorizer(ngram_range=(1,1))
vectors = vectorizer.fit_transform(l)

tfidf = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
tfidf

,000,100,108,10th,113,11th,1200,12th,1399,13th,1431,14th,1500,1509,1586,15th,16th,1700,1726,1778,1790,1791,1792,1801,1802,1829,1849,1859,1861,1864,1868,1879,1880,18th,1900,1910,1930,1935,19th,1st,...,youth,youthful,youthfulness,yr,yulian,yushin,yust,yusupov,zada,zaharovitch,zametov,zamm,zapiski,zara,zarnitsyn,zeal,zealand,zealous,zealously,zebra,zed,zeitschrift,zenith,zephyr,zero,zest,zeus,zhukovsky,zigzag,zigzagged,zillah,zimmerman,zimoveykin,zinc,zinovy,zone,zoomed,zorn,zossimov,zverkov
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Doc2Vec
documents = [TaggedDocument(s, [i]) for i, s in enumerate(s)]
model = Doc2Vec(documents)

doc2vec = pd.DataFrame([list(model[document]) for document in range(len(documents))])
doc2vec

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.009061,0.001650,-0.017766,0.011020,-0.006357,-0.013010,0.009174,0.009206,-0.008814,0.018193,0.020498,0.009504,0.017460,0.013335,-0.011484,-0.008979,-0.025746,0.011176,0.006338,-0.019224,-0.014850,0.001947,0.001638,-0.000514,-0.009903,0.013234,-0.012143,0.004342,0.020600,-0.007373,0.004842,0.004529,-0.021188,0.019659,0.039027,-0.004741,-0.005525,-0.036917,0.002566,0.002390,...,-0.014674,-0.032616,-0.000420,0.004515,-0.002422,0.019849,0.019290,0.009618,0.003688,0.024839,-0.005464,-0.020398,0.001902,0.013154,-0.014971,0.026068,0.005149,0.027462,-0.001992,0.028670,0.009099,-0.039653,-0.014742,-0.013391,-0.008517,-0.004505,0.007271,-0.021299,0.019098,0.003609,-0.023964,-0.012596,-0.001709,-0.004755,-0.004337,-0.026712,0.000666,-0.008511,-0.001237,0.004330
1,-0.001703,0.011254,-0.011162,0.002532,-0.006941,-0.006755,0.007361,0.001174,-0.000148,0.008013,0.010506,-0.002449,0.010110,0.006111,-0.009130,-0.005072,-0.015779,0.004779,0.004870,-0.009915,-0.011910,0.000686,0.001535,0.003566,-0.007023,-0.000159,0.000439,0.005141,0.008093,-0.007652,0.005202,0.003925,-0.007325,0.014740,0.016992,-0.006077,0.000070,-0.017553,-0.002109,0.001904,...,-0.002669,-0.011842,0.002347,0.004457,-0.003181,0.006221,0.016707,0.008140,-0.003080,0.019167,-0.001813,-0.007748,0.000420,0.002244,-0.007372,0.011816,0.003928,0.010123,0.001505,0.019202,0.005459,-0.010917,-0.004025,-0.003099,0.003603,0.005810,0.009239,-0.009238,0.006739,0.002723,-0.011846,-0.004821,0.000340,0.000050,0.002206,-0.012670,-0.005234,-0.010986,0.000232,0.008160
2,-0.001103,0.002608,-0.004411,0.003938,-0.012416,-0.005163,-0.001447,0.005378,0.004965,0.010570,0.007032,-0.011041,0.010071,0.007817,-0.001709,-0.000866,-0.012741,0.010897,0.002511,-0.004846,-0.011798,-0.001220,-0.006765,0.007386,-0.001915,0.003627,-0.000409,-0.003086,0.003224,-0.000363,0.006245,-0.001978,-0.014507,0.013368,0.021968,-0.005857,-0.002517,-0.017624,-0.001509,0.007778,...,-0.011790,-0.006897,-0.002673,0.004487,-0.005616,0.007103,0.017469,0.007930,-0.005799,0.018184,-0.001596,-0.005747,0.009040,0.012755,-0.006481,0.013735,-0.001521,0.016469,0.001191,0.011041,0.007094,-0.011408,-0.000541,-0.001441,0.001898,0.008495,0.003867,-0.003102,-0.000456,-0.002149,-0.004193,-0.005823,0.001158,0.000800,0.007531,-0.009670,-0.005099,-0.014718,-0.000197,0.003223
3,-0.017369,-0.031731,-0.037247,0.017462,0.011488,-0.017586,0.027984,0.048967,-0.001022,0.018295,0.046203,0.043095,0.004796,0.026762,-0.037469,-0.043497,-0.038040,0.009988,-0.021755,-0.057625,0.030409,-0.012297,0.064838,-0.047568,0.018251,0.014312,-0.032907,0.008854,0.034492,0.016277,-0.039997,0.009046,-0.027941,0.042296,0.003776,0.017330,-0.000589,-0.052468,0.009338,0.005969,...,0.001357,-0.058210,0.032059,-0.001669,0.033291,0.039239,0.000596,-0.005198,0.010489,0.007451,0.001194,-0.014291,-0.036537,0.000620,-0.009821,0.003573,0.009608,0.034353,0.006171,0.015393,0.005341,-0.100185,-0.020315,0.015385,-0.016422,-0.061363,0.028619,-0.015139,0.052879,0.042344,-0.032116,-0.003156,-0.029376,0.002215,-0.003567,-0.069775,0.039539,0.051985,0.008497,-0.011882
4,-0.013042,-0.051992,0.022482,0.036875,0.032228,0.014418,0.021817,0.021230,-0.032264,-0.022992,0.009654,0.034577,-0.061472,0.014536,0.042159,-0.020490,0.028086,0.022984,-0.029828,0.055695,0.061884,-0.048877,0.027212,0.017399,0.045098,-0.006933,0.002562,-0.025263,-0.003396,-0.045478,0.024409,0.041145,0.006118,0.050610,0.045910,-0.005354,0.062934,0.009650,-0.019817,-0.010263,...,0.084831,-0.026790,-0.019116,0.013622,0.016932,0.031134,0.038454,-0.009625,-0.026220,0.033093,0.030259,0.017178,0.034552,0.035050,0.021908,0.008174,0.054597,-0.054349,0.002728,-0.060363,0.004527,0.006565,-0.025572,0.052689,-0.001008,-0.007052,0.051579,-0.000976,-0.104534,-0.019863,0.000870,-0.051509,0.060540,0.004026,-0.004237